In [78]:
import pandas as pd
out=pd.read_table('csdn_blog_100.csv',sep='\t',
                engine='python',index_col=['_id'],header=None,names=['_id','标题','标签','时间','pv','作者','原创','c'])
out['tfkey'] = ''
out['trkey'] = ''
out['nlp']=''

In [79]:
import jieba
import jieba.analyse
import jieba.posseg as pseg
from bs4 import BeautifulSoup
from gensim import corpora, models, similarities

wordmodel = jieba.analyse
# 加停用词库
wordmodel.set_stop_words(stop_words_path='../words/CNstop_words_zh.txt')
import codecs
stopwords = codecs.open('../words/CNstop_words_zh.txt', 'r', encoding='utf8').readlines()
stopwordlist = [w.strip() for w in stopwords]
# 结巴分词后的停用词性 [标点符号、连词、助词、副词、介词、时语素、‘的’、数词、方位词、代词]
stop_flag = ['x', 'c', 'u','d', 'p', 't', 'uj', 'm', 'f', 'r']

In [80]:
def genkeyword(doc,W=5,K=5):
    """
    取关键词
    :param doc:             文本内容
    :param stop_flag:       停用词属性
    :param W:               textrank窗口磊小
    :param K:               topK取词
    :return:                2个关键词列表， tfidf, textrank
    """
    resulttfidf = []
    resulttextrank = []
    resulttfidf = []
    resulttextrank = []
    # tfidf
    tfidfwords = wordmodel.extract_tags(doc, topK=K, withWeight=False,allowPOS=['ns','n','vn','v','nr','x'])

    # textrank
    wordmodel.TextRank().span = W
    textrankwords = wordmodel.textrank(doc, topK=K, withWeight=False,allowPOS=['ns','n','vn','v','nr','x'])
    return tfidfwords, textrankwords

def tokenization(doc,sw):
    """
    分词
    :param doc:         文本内容
    :param sw:          停用词列表
    :return:            分词后的列表
    """
    result = []
    text = doc
    words = pseg.cut(text)
    for word, flag in words: 
        if flag not in stop_flag and word not in sw: 
            result.append(word)
    return result

ids=[]
dockeys=[]
# 100篇文档分词，保存
for row in out.itertuples():
    soup = BeautifulSoup(row.c,'html5lib').getText().strip()
    ids.append(row.Index)
    word_list = tokenization(soup,stopwordlist)
    dockeys.append(word_list)
    out.loc[out.index==row[0],['nlp']] = ' '.join([w for w in word_list])
    tfwords,trwords = genkeyword(soup, W=5, K=8)
    out.loc[out.index == row[0], ['tfkey']] = ' '.join([w for w in tfwords])
    out.loc[out.index == row[0], ['trkey']] = ' '.join([w for w in trwords])

# out.drop(out.columns[[6]], axis=1,inplace=True)
# out.to_csv('csdn_blog_100_nlp_tr_tf_keyword.csv',sep='\t')
# try:
# #     保存文档id列表
#     import pickle
#     f = open("./csdn_blog_ids_100.dat", "wb")
#     pickle.dump(ids, f)
#     f.close()
# except Exception as e:
#     print('csdn_blog_ids_100.dat保存文件出错')
    
# try:
# #     保存文档dockeys列表
#     import pickle
#     f = open("./csdn_blog_dockeys_100.dat", "wb")
#     pickle.dump(dockeys, f)
#     f.close()
# except Exception as e:
#     print('csdn_blog_dockeys_100.dat保存文件出错')

In [82]:
out.drop(out.columns[[2,3,4,5,6,9]], axis=1,inplace=True)
out

,标题,标签,tfkey,trkey
_id,,,,
321,WhatisthenextBIGthing?,internet 产品 apple blackberry nokia 服务器,产品 应用 技术 市场 设备 推出 领域 厂商,市场 产品 领域 技术 应用 没有 设备 计算
322,关于我在中华读书报发表的文章,读书 出版 教育 设计模式 语言 java,教材 书籍 教学 技术 课堂 学生 教科书 习题,教材 技术 书籍 教学 学生 方法 知识 市场
638,LaTeX常用功能,report,设置 页眉 字号 缩进 页脚 版心 包来 行距,设置 显示 缩进 页脚 包来 纸张 习惯 字号
823,成功项目的杀手：领导意志,工作 企业应用 算法 申诉 微软 浏览器,项目 软件 队伍 领导 油田 分数 系统 考核,项目 队伍 领导 软件 油田 系统 企业 公司
1690,腾讯上市马化腾6年创业身家9亿,腾讯 创业 qq 互联网 网络 游戏,移动梦网 马化腾 服务 用户 业务 互联网 增值 网络广告,业务 服务 马化腾 互联网 用户 注册 收入 移动梦网
1796,医院信息化发展调查研究报告（来自赛迪）,医疗 服务器 技术人 网络 产品 internet,医院 信息系统 服务器 信息化 建设 医疗 系统 选型,医院 信息系统 系统 建设 信息化 医疗 服务器 管理
1823,华为的冬天(任正非）,华为 工作 生活 报表 数据库 制造,干部 改进 流程 批判 管理 变革 公司 效益,干部 公司 管理 不能 流程 批判 改进 没有
1826,软件公司创业完全手册[一],创业 工作 生活 融资 平台,创业 软件 发展 过程 达到 人生 分配 园区,软件 创业 发展 过程 速度 产业 分配 希望
1834,J2ME (Java 2 Micro Edition) 简介,java j2me 嵌入式 平台 j2se linux,程序 类别 函式 平台 执行 审核 装置 预先,程序 平台 类别 执行 函式 审核 装置 预先


In [19]:
# from gensim import corpora, models, similarities
# # 建立TF-IDF模型
# # 建立词袋模型
# dictionary = corpora.Dictionary.load('csdn_blog_100.dict')
# # 生成词向量
# corpus = corpora.MmCorpus('csdn_blog_100.mm')
# tfidf = models.TfidfModel.load("./csdn_blog_model_100.tfidf")
# tfidf_corpus = tfidf[corpus]

In [55]:
# # 构建LDA模型，设置主题数为10
# lda = models.LdaModel(corpus, id2word=dictionary, num_topics=10)
# # lsi.print_topics(4)
# lda_corpus = lda[corpus]
# # 生成lda模型,相似度矩阵
# index_lda = similarities.MatrixSimilarity(lda_corpus)
# # index_lda.save('csdn_blog_100_lda.index')

In [83]:
# from operator import itemgetter, attrgetter
# f = 0.7
# K = 8
# for c in lda_corpus:
#     doc_topics = sorted(c, key=itemgetter(1),reverse=True)
#     fv=0
#     doc_f_topics = []
#     for t in doc_topics:
#         fv+=t[1]
#         doc_f_topics.append(t)
#         if fv >f:
#             break
            
#     for d in doc_f_topics:
#         topwords = lda.get_topic_terms(d[0])
#         print(d[0],topwords)